# Example: 2D Simulation

In [1]:
using UltraDark
using Test
using NPZ
using CairoMakie
using CSV

Threads.nthreads()

1

## Define initial conditions

Define a 2D grid.

In [2]:
resol = 128
len = 10.0

grids = Grids((len, len, len / resol), (resol, resol, 1));

Add some solitons to the grid.  Strictly speaking, these aren't solitons in 2D, but they'll do for demonstration purposes.

In [3]:
mass = 10
position_1 = [-len / 5, -len / 5, 0]
position_2 = [-len / 5, +len / 5, 0]
velocity = [1, 0, 0]
phase_1 = 0
phase_2 = π
t0 = 0

UltraDark.Initialise.add_fdm_soliton!(grids, mass, position_1, velocity, phase_1, t0)
UltraDark.Initialise.add_fdm_soliton!(grids, mass, position_2, velocity, phase_2, t0)

## Set options

In [4]:
output_dir = joinpath(mktempdir(), "output", "2D")

output_times = 0:0.1:5
output_config = OutputConfig(output_dir, output_times);

options = Config.SimulationConfig();

## Run simulation

In [5]:
@time simulate!(grids, options, output_config)

[ Info: Reached time 0.09999999999999998
[ Info: Reached time 0.20000000000000004
[ Info: Reached time 0.30000000000000004
[ Info: Reached time 0.4
[ Info: Reached time 0.5
[ Info: Reached time 0.5999999999999999
[ Info: Reached time 0.6999999999999997
[ Info: Reached time 0.8000000000000002
[ Info: Reached time 0.9
[ Info: Reached time 0.9999999999999999
[ Info: Reached time 1.1
[ Info: Reached time 1.2000000000000002
[ Info: Reached time 1.3000000000000003
[ Info: Reached time 1.4000000000000004
[ Info: Reached time 1.5000000000000004
[ Info: Reached time 1.6000000000000005
[ Info: Reached time 1.7000000000000006
[ Info: Reached time 1.8000000000000007
[ Info: Reached time 1.8999999999999997
[ Info: Reached time 1.9999999999999998
[ Info: Reached time 2.1000000000000014
[ Info: Reached time 2.2
[ Info: Reached time 2.2999999999999994
[ Info: Reached time 2.400000000000001
[ Info: Reached time 2.5
[ Info: Reached time 2.6000000000000014
[ Info: Reached time 2.7
[ Info: Reached time 2.

## Plot output

In [6]:
summary = CSV.File(joinpath(output_config.directory, "summary.csv"));

rho_last = npzread("$(output_config.directory)/rho_$(length(output_times)).npy");
δ_lims = extrema(rho_last .- 1)

fig_anim = Figure()
ax_anim = Axis(fig_anim[1, 1], aspect = DataAspect())

hidedecorations!(ax_anim)

cb_density = Colorbar(fig_anim[1, 2], limits = δ_lims, label = L"$\rho/\rho_{\text{crit}}$")

frame = Observable(1)

rho = lift(i -> npzread(joinpath(output_config.directory, "rho_$(i).npy"))[:, :, 1], frame)
δ = @lift($rho .- 1)

contourf!(ax_anim, grids.x[:, 1, 1], grids.y[1, :, 1], δ, levels = range(-1, δ_lims[2], 10))

record(fig_anim, "2d.mkv", 1:length(output_times); framerate = 5) do f
    frame[] = f
end

"2d.mkv"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*